#### Import libraries and authenticate GEE

In [1]:
import os
import ee
import geemap

In [67]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWisi8BwXENXqM8_xDNCI8T3asXtCkFPRM_Hb0KjGPctbiL6mlj97Eo

Successfully saved authorization token.


## Select one ward and display the SANLC 2020 in its location

In [3]:
Map = geemap.Map(center=(-33.918861, 18.423300))
# Upload the shapefile of the ROI(SA)
#sa  = ee.FeatureCollection("users/noecareme/Census_2011_Spatial_Geography")
sa    = geemap.shp_to_ee('/Users/noe/Documents/PostDoc/Workshop/Day 3/SA4/ADM4_PCODE_ZA5950090.shp')
Map.addLayer(sa, {}, 'ROI')
Map

Map(center=[-33.918861, 18.4233], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [4]:
### Legend and colors dictionary
dico = {"names": ["Contiguous (indigenous) Forest (combined very high, high, medium)", "Contiguous Low Forest & Thicket (combined classes)", "Dense Forest & Woodland (35 - 75% cc)", "Open Woodland (10 - 35% cc)", "Contiguous & Dense Planted Forest (combined classes)", "Open & Sparse Planted Forest", "Temporary Unplanted Forest", "Low Shrubland (other regions)", "Low Shrubland (Fynbos)", "Low Shrubland (Succulent Karoo)", "Low Shrubland (Nama Karoo)", "Sparsely Wooded Grassland (5 - 10% cc)", "Natural Grassland", "Natural Rivers", "Natural Estuaries & Lagoons", "Natural Ocean, Coastal", "Natural Lakes", "Natural Pans (flooded @ obsv time)", "Artificial Dams (incl. canals)", "Artificial Sewage Ponds", "Artificial Flooded Mine Pits", "Herbaceous Wetlands (currently mapped)", "Herbaceous Wetlands (previous mapped extent)", "Mangrove Wetlands", "Natural Rock Surfaces", "Dry Pans", "Eroded Lands", "Sand Dunes (terrestrial)", "Coastal Sand Dunes & Beach Sand", "Bare Riverbed Material", "Other Bare", "Cultivated Commercial Permanent Orchards", "Cultivated Commercial Permanent Vines", "Cultivated Commercial Sugarcane Pivot Irrigated", "Commercial Permanent Pineapples", "Cultivated Commercial Sugarcane Non-Pivot (all other)", "Cultivated Emerging Farmer Sugarcane Non-Pivot (all other)", "Commercial Annuals Pivot Irrigated", "Commercial Annuals Non-Pivot Irrigated", "Commercial Annuals Crops Rain-Fed / Dryland / Non-Irrigated", "Subsistence / Small-Scale Annual Crops", "Fallow Land & Old Fields (Trees)", "Fallow Land & Old Fields (Bush)", "Fallow Land & Old Fields (Grass)", "Fallow Land & Old Fields (Bare)", "Fallow Land & Old Fields (Low Shrub)", "Residential Formal (Tree)", "Residential Formal (Bush)", "Residential Formal (low veg / grass)", "Residential Formal (Bare)", "Residential Informal (Tree)", "Residential Informal (Bush)", "Residential Informal (low veg / grass)", "Residential Informal (Bare)", "Village Scattered (bare only)", "Village Dense (bare only)", "Smallholdings (Tree)", "Smallholdings (Bush)", "Smallholdings (low veg / grass)", "Smallholdings (Bare)", "Urban Recreational Fields (Tree)", "Urban Recreational Fields (Bush)", "Urban Recreational Fields (Grass)", "Urban Recreational Fields (Bare)", "Commercial", "Industrial", "Roads & Rail (Major Linear)", "Mines: Surface Infrastructure", "Mines: Extraction Sites: Open Cast & Quarries combined", "Mines: Extraction Sites: Salt Mines", "Mines: Waste (Tailings) & Resource Dumps", "Land-fills", "Fallow Land & Old Fields (wetlands)"], "colors": ["#F2F2F2", "#065106", "#005F00", "#008500", "#F74006", "#F9764D", "#F9906C", "#B8ABD1", "#8FAB39", "#AC92C5", "#AC9CDA", "#85D285", "#D2B485", "#00009F", "#041FA7", "#0639AB", "#0D50AC", "#125FAC", "#1373B4", "#1D81B6", "#1F8EB8", "#06DEDC", "#06E0D0", "#9F1FEC", "#ffffe0", "#DCDAC5", "#F9E0E0", "#F9F9C5", "#F9F9A7", "#CDD2E0", "#ffffe0", "#A62C39", "#B31F5C", "#DB0000", "#9F3978", "#FF0000", "#F64D6C", "#381A12", "#521F1C", "#85402C", "#C5735F", "#C1436C", "#C55E82", "#D27592", "#E0AAB8", "#DB90A9", "#ECDB0F", "#F6EC13", "#F9F81F", "#FFFF29", "#EC82EC", "#F691E0", "#F99FCF", "#FFC5CF", "#ECC500", "#FFD91F", "#AC7879", "#B89192", "#C49C9E", "#D2B8B8", "#BFFF00", "#33FF33", "#66FF66", "#99FF99", "#C49F0D", "#8F8506", "#F9DD03", "#FFFF00", "#B30606", "#C50606", "#D21D1A", "#F95479", "#6CE7DC"]}

sanlcd2020 = ee.Image('users/noecareme/SA_NLC_2020_GEO').select('b1').clip(sa)
Map.addLayer(sanlcd2020, {'min':1, 'max':73,'palette':dico['colors']}, 'SANLC2020')
Map

Map(bottom=616289.0, center=[-30.008691063294773, 30.903715019905704], controls=(WidgetControl(options=['posit…

In [5]:
#### Add legend to the map
Map.add_legend(legend_keys=dico['names'], legend_colors=dico['colors'], position='topright',padding = '8px 15px')
Map 

Map(bottom=616289.0, center=[-30.008691063294773, 30.903715019905704], controls=(WidgetControl(options=['posit…

In [6]:
sanlcd2020.getInfo()

{'type': 'Image',
 'bands': [{'id': 'b1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [92927, 71248],
   'crs': 'EPSG:4326',
   'crs_transform': [0.00017966305682390397,
    0,
    16.350236486259412,
    0,
    -0.00017966305682390397,
    -22.12406814340922]}],
 'version': 1633513819831825,
 'id': 'users/noecareme/SA_NLC_2020_GEO',
 'properties': {'system:asset_size': 639000421}}

# Extract data from each class from the above ward displayed

In [19]:
EO =[]
for i in range(1,74):
    crop20 = sanlcd2020.eq(i)
    areaImage20 = crop20.multiply(ee.Image.pixelArea())
    area20 = areaImage20.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': sa.geometry(),
      'scale': 20,
      'maxPixels': 3e10
      })
    classAreaSqKm20 = ee.Number(area20.get('b1')).divide(1e6)
    #EA20 = ee.Feature(None, {'Area': classAreaSqKm20})
    EO.append(classAreaSqKm20.getInfo())
pd.DataFrame(EO).to_excel('/Users/noe/Documents/Data_science/Geospatial_DS/SANLC_Data_Collection/sanlc/5950090.xlsx')#, index=[0])

## Loop through all the wards in South Africa and collect land cover

In [14]:
import pandas as pd
IDSS = pd.read_excel('/Users/noe/Documents/Data_science/Geospatial_DS/SANLC_Data_Collection/Excelfiles/WARD_ID.xlsx')
IDSS = list(IDSS[0])
col_names = [int(i) for i in IDSS]

In [99]:
for j in col_names:
    table = geemap.shp_to_ee("/Users/noe/Documents/Data_science/Geospatial_DS/SANLC_Data_Collection/SA_LEVEL4/ADM4_PCODE_ZA"+str(j)+".shp")

    EO =[]
    for i in range(1,74):
        crop20 = sanlcd2020.eq(i)
        areaImage20 = crop20.multiply(ee.Image.pixelArea())
        area20 = areaImage20.reduceRegion(**{
          'reducer': ee.Reducer.sum(),
          'geometry': table,
          'scale': 20,
          'maxPixels': 3e10
          })
        classAreaSqKm20 = ee.Number(area20.get('b1')).divide(1e6)
        #EA20+str(i) = ee.Feature(None, {'Area': cropAreaSqKm20+str(i)})
        EO.append(classAreaSqKm20)
    
    pd.DataFrame(EO).to_excel('/Users/noe/Documents/Data_science/Geospatial_DS/SANLC_Data_Collection/sanlc/'+str(j)+'.xlsx')#, index=[0])

### Some more datasets on South Africa
- [South Africa-healthsites](https://data.humdata.org/dataset/south-africa-healthsites);
- [GRID3 South Africa Settlement Extents, Version 01.01](https://data.humdata.org/dataset/grid3-south-africa-settlement-extents-version-01-01);

- [South Africa - Population Density](https://data.humdata.org/dataset/worldpop-population-density-for-south-africa);
- [South Africa - Population Counts](https://data.humdata.org/dataset/worldpop-population-counts-for-south-africa);
- [HOTOSM South Africa Roads(OpenStreetMap Export)](https://data.humdata.org/dataset/hotosm_zaf_roads).
